In [1]:
from keras.utils import normalize
from keras.models import load_model
import cv2
from PIL import Image
import numpy as np
from patchify import patchify
import os

In [2]:
img = cv2.imread(r"D:\Research\Isaacs Lab\Charles Project\Batch Images\21G-20_40X_02_0002.png", 0)

In [3]:
patch_size=256

In [4]:
model = load_model(r"D:\Research\Isaacs Lab\DeepAxon\models\model1.keras")

In [5]:
SIZE_X = img.shape[0] // patch_size * patch_size
SIZE_Y = img.shape[1] // patch_size * patch_size

In [6]:
img = Image.fromarray(img)
img = img.crop((0,0,SIZE_Y, SIZE_X))
img = np.array(img)

In [7]:
patches = patchify(img, (patch_size, patch_size), patch_size//2)

In [8]:
def get_pos(shape, i,j):
    i_max = shape[0]-1
    j_max = shape[1]-1
    if i == 0 and j == 0:
        pos = 0
    elif i == 0 and j == j_max:
        pos = 2
    elif i == i_max and j == 0:
        pos = 6
    elif i == i_max and j == j_max:
        pos = 8
    elif i == 0:
        pos = 1
    elif i == i_max:
        pos = 7
    elif j == 0:
        pos = 3
    elif j == j_max:
        pos = 5
    else:
        pos = 4
    return pos

In [9]:
def hann_fn(x):
    return (1 - np.cos(2 * np.pi * x / 255))/2

In [10]:
def hann_window(pos):
    i, j = np.meshgrid(np.arange(256), np.arange(256), indexing='ij')
    condition1 = (i <= 128) & (j <= 128)
    condition2 = (i > 128) & (j < 128)
    condition3 = (i < 128) & (j > 128)
    condition4 = ~condition1 & ~condition2 & ~condition3
    
    scaler = np.zeros((256,256), dtype=float)
    if pos == 0:
        scaler[condition1] = 1
        scaler[condition2] = hann_fn(i[condition2])
        scaler[condition3] = hann_fn(j[condition3])
        scaler[condition4] = hann_fn(i[condition4]) * hann_fn(j[condition4])
    elif pos == 1:
        scaler[condition1] = hann_fn(j[condition1])
        scaler[condition2] = hann_fn(i[condition2]) * hann_fn(j[condition2])
        scaler[condition3] = hann_fn(j[condition3])
        scaler[condition4] = hann_fn(i[condition4]) * hann_fn(j[condition4])
    elif pos == 2:
        scaler[condition1] = hann_fn(j[condition1])
        scaler[condition2] = hann_fn(i[condition2]) * hann_fn(j[condition2])
        scaler[condition3] = 1
        scaler[condition4] = hann_fn(i[condition4])
    elif pos == 3:
        scaler[condition1] = hann_fn(i[condition1])
        scaler[condition2] = hann_fn(i[condition2])
        scaler[condition3] = hann_fn(i[condition3]) * hann_fn(j[condition3])
        scaler[condition4] = hann_fn(i[condition4]) * hann_fn(j[condition4])
    elif pos == 4:
        scaler[condition1] = hann_fn(i[condition1]) * hann_fn(j[condition1])
        scaler[condition2] = hann_fn(i[condition2]) * hann_fn(j[condition2])
        scaler[condition3] = hann_fn(i[condition3]) * hann_fn(j[condition3])
        scaler[condition4] = hann_fn(i[condition4]) * hann_fn(j[condition4])
    elif pos == 5:
        scaler[condition1] = hann_fn(i[condition1]) * hann_fn(j[condition1])
        scaler[condition2] = hann_fn(i[condition2]) * hann_fn(j[condition2])
        scaler[condition3] = hann_fn(i[condition3])
        scaler[condition4] = hann_fn(i[condition4])
    elif pos == 6:
        scaler[condition1] = hann_fn(i[condition1])
        scaler[condition2] = 1
        scaler[condition3] = hann_fn(i[condition3]) * hann_fn(j[condition3])
        scaler[condition4] = hann_fn(j[condition4])
    elif pos == 7:
        scaler[condition1] = hann_fn(i[condition1]) * hann_fn(j[condition1])
        scaler[condition2] = hann_fn(j[condition2])
        scaler[condition3] = hann_fn(i[condition3]) * hann_fn(j[condition3])
        scaler[condition4] = hann_fn(j[condition4])
    elif pos == 8:
        scaler[condition1] = hann_fn(i[condition1]) * hann_fn(j[condition1])
        scaler[condition2] = hann_fn(j[condition2])
        scaler[condition3] = hann_fn(i[condition3])
        scaler[condition4] = 1
        
    return scaler

In [11]:
def patch_predict(patches):
    pred_img = np.zeros(img.shape)
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            patch = patches[i,j,:,:]
            patch = normalize(patch)
            patch = np.expand_dims(patch, axis=(0,3))
            pred = model.predict(patch)
            pred = np.argmax(pred, axis=3)
            pred = pred[0,:,:]
            
            patch_pos = get_pos(patches.shape, i, j)
            hann_matrix = hann_window(patch_pos)
            adj_pred = pred * hann_matrix
            
            i_start = i*128
            i_end = i_start+256
            j_start = j*128
            j_end = j_start+256
            pred_img[i_start:i_end, j_start:j_end] += adj_pred
    pred_img = np.round(pred_img).astype(int)
    return pred_img

In [12]:
pred = patch_predict(patches)

1/1 [==============================] - 0s 162ms/step


In [13]:
colors = {
    0: (0, 0, 0),    # Black for 0
    1: (128, 128, 128),  # Middle gray for 1
    2: (255, 255, 255),  # White for 2
}

In [14]:
colored_array = np.zeros((pred.shape[0], pred.shape[1], 3), dtype=np.uint8)

In [15]:
for value, color in colors.items():
    colored_array[pred == value, :] = color

In [18]:
np.unique(colored_array)

array([  0, 128, 255], dtype=uint8)

In [16]:
image = Image.fromarray(colored_array)

In [17]:
image.save(r"D:\Research\Isaacs Lab\DeepAxon\test\images\segmented\21G-20_40X_02_0002.png")